In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [2]:
model_local = ChatOllama(model="mistral",temperature=0)

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\RAHUL\Desktop\chatbot_new\chatbot\ilovepdf_merged.pdf")
pages = loader.load_and_split()

In [4]:
pages

[Document(page_content='2022-2023', metadata={'source': 'C:\\Users\\RAHUL\\Desktop\\chatbot_new\\chatbot\\ilovepdf_merged.pdf', 'page': 0}),
 Document(page_content='STAY', metadata={'source': 'C:\\Users\\RAHUL\\Desktop\\chatbot_new\\chatbot\\ilovepdf_merged.pdf', 'page': 2}),
 Document(page_content='Affiliated to Multiple Scler osis Inter national F ederation (MSIF)\nwww.msif.orgOfﬁce: 87-A, Ground Floor, DDA Flats, Shahpur Jat, New Delhi-110049\nTel: 91-11-26490087, Mobile: 9871596452\nWebsite: www.mssocietyindia.org\nBipasha Gupta\nHony . Chairperson\nMeenakshi Bhujwala\nHony . TreasurerRagini Mathur\nHony . Joint Secretar yMeera Pinky Hora\nExecutive MemberMridula Murgai\nHony . Vice ChairpersonPrabal Malaker\nHony . Secretar y\nPraveen Khetarpal\nExecutive MemberAmeeta Gupta\nExecutive MemberNina K ochhar\nExecutive Member\nRaj Kumar Shashi V erma Shagufta F atima Raji Vijayen Puja Chowdhar y\n Gopal Chakrabor ty', metadata={'source': 'C:\\Users\\RAHUL\\Desktop\\chatbot_new\\chatbo

In [5]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=100)
doc_splits = text_splitter.split_documents(pages)

In [8]:
import os
import uuid
import base64
from IPython import display
from unstructured.partition.pdf import partition_pdf
# from langchain.chat_models import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_community import embeddings

# from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [11]:
# vectorstore = Chroma.from_documents(
#             documents=doc_splits,
#             collection_name="rag-chroma",
#             embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
#         )
# retriever = vectorstore.as_retriever()
# embeddings = embeddings.OllamaEmbeddings(model='nomic-embed-text')
from langchain_community.vectorstores import FAISS
embedding = embeddings.OllamaEmbeddings(model='nomic-embed-text')

rec_vectorstore = FAISS.from_documents(documents=doc_splits, embedding=embedding)
print(rec_vectorstore.index.ntotal)

rec_vectorstore.save_local("demo_ms_")

1303


Before RAG
---

In [44]:
# 3. Before RAG
print("Before RAG\n")
before_rag_template = "what is {topic}"
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()

Before RAG



In [45]:
print(before_rag_chain.invoke({"topic": "analog?"}))

 In the context of electronics and communication, "analog" refers to a continuous signal or system where information is represented by physical quantities that can vary continuously over time or space. This is in contrast to digital systems, where information is represented as discrete binary values (0s and 1s).

Analog signals can take various forms such as voltage levels, current flow, or even mechanical displacement. Examples of analog signals include an audio waveform from a microphone, temperature readings from a thermometer, or the position of a joystick in a video game. Analog systems process these signals using various components like amplifiers, filters, and modulators to manipulate and transmit the information they carry.

Analog technology has been widely used for decades due to its ability to handle complex and dynamic data, but with the advent of digital technology, many applications have shifted towards digital processing for improved accuracy, reliability, and ease of tr

In [32]:
print(before_rag_chain.invoke({"topic": "cancer spread?"}))

 Cancer spreads through the process of metastasis. During this process, cells from the primary tumor break away and travel to other parts of the body. These cells can then form new tumors in these locations, leading to the development of metastatic lesions.

There are several ways that cancer cells can spread:

1. Through the bloodstream: Cancer cells can enter the bloodstream through the lymphatic system or directly from a blood vessel near the primary tumor. Once in the bloodstream, these cells can travel to any part of the body and form new tumors.
2. Through the lymphatic system: The lymphatic system is a network of vessels that helps to filter out waste and bacteria from the body. Cancer cells can enter the lymphatic system near the primary tumor and travel to other parts of the body, forming new tumors in the lymph nodes or other organs.
3. Through direct invasion: Some cancers can grow into nearby structures and invade them directly, spreading cancer cells to other areas of the 

In [49]:
print(before_rag_chain.invoke({"topic": "anastomosis?"}))

 An anastomosis is a surgical connection between two ends of tubular structures, such as intestines or blood vessels. It allows the continuity of the flow of substances, like digestive contents or blood, to be restored after a section has been removed due to injury or disease. The term comes from the Greek word "anastomosis," which means "re-establishing a connection." Anastomoses can be performed using various techniques, including suturing, stapling, or the use of surgical couplers or grafts.


In [13]:

print(before_rag_chain.invoke({"topic": "alpha blocker"}))

Before RAG

 An alpha blocker is a type of medication used to treat various conditions related to the narrowing or tightening of blood vessels. The primary action of alpha blockers is to block the effects of certain natural chemicals, called alpha-adrenergic agonists, on the body.

These medications are commonly prescribed for the following conditions:

1. Hypertension (high blood pressure): Alpha blockers help relax and widen the blood vessels, making it easier for blood to flow through them and reducing the pressure in the arteries.
2. Benign prostatic hyperplasia (BPH): In this condition, an enlarged prostate gland can obstruct the flow of urine from the bladder. Alpha blockers help relax the muscles in the prostate and bladder neck, making it easier to urinate.
3. Raynaud's phenomenon: This is a condition where the small blood vessels in the fingers and toes constrict, reducing blood flow and causing coldness, pain, and discoloration. Alpha blockers help improve blood flow to these

In [14]:
before_rag_chain.invoke({"topic": "analog"})

' In the context of electronics and communication, an "analog" system or signal represents information as a continuous waveform or stream of data, as opposed to digital systems that represent data as discrete binary values.\n\nAnalog signals can take various forms such as voltage levels, current flow, or even mechanical displacement (like in the case of sound waves or seismic activity). These signals can be processed and transmitted using analog circuits, which manipulate the continuous waveform to perform operations like amplification, filtering, or modulation.\n\nExamples of analog systems include:\n\n1. Telephone lines: Voice calls are transmitted as continuous electrical signals over telephone lines.\n2. Radio broadcasts: AM and FM radio signals carry audio information as continuous waves.\n3. Video signals: Television and movie images are represented as continuous electrical or optical signals.\n4. Analog computers: Early computing devices used analog circuits to perform mathemati

In [16]:
before_rag_chain.invoke({"topic": "anesthesia"})

" Anesthesia is a medical treatment that helps to prevent or reduce the sensation of pain during a medical procedure. It can also help to induce and maintain unconsciousness, which is called general anesthesia. There are different types of anesthesia, including local anesthesia, which numbs a specific area of the body; regional anesthesia, which numbs a larger area; and general anesthesia, which affects the entire body and causes a deep sleep-like state. Anesthesia is typically administered by an anesthesiologist or nurse anesthetist, who monitors the patient's vital signs and adjusts the level of anesthesia as needed to ensure the patient's safety during the procedure."

After RAG
---

In [54]:
# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """provide the answer for the question only from this given context:
{context}
Question: {question} and only give the answer from the given context
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)



########
After RAG



In [47]:
chunks = []
async for chunk in after_rag_chain.astream("What is analog?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 An analog is a substance that is similar, but not identical to another substance. For example, a man-made version of a naturally occurring substance.

In [48]:
chunks = []
async for chunk in after_rag_chain.astream("What is anastomosis?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 Anastomosis is a surgical procedure in which a diseased part of a tubular body structure is removed, and the ends of healthy sections are reconnected.

In [55]:
chunks = []
async for chunk in after_rag_chain.astream("What is the future of cancer screening?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 The future of cancer screening involves the use of advanced technologies such as expression profiling, proteomics, nanotechnology, and hyperlinks to early detection and personalized treatment. Expression profiling allows for determination of relative output of hundreds or thousands of molecules in cells, providing valuable information about cancer behavior and response to drugs. Proteomic methods compare the relative amounts of many proteins to distinguish more aggressive cancers and predict drug responses. Nanotechnology produces tiny particles for accurate tumor imaging and targeted drug delivery. Hyperlinks refer to further resources or information on these topics.

In [33]:
chunks = []
async for chunk in after_rag_chain.astream("What is the cancer stage?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 The cancer stage is determined by examining tissue samples from surgery or a biopsy under a microscope, using systems like the TNM staging system to assess the size of the tumor, whether it has spread to nearby lymph nodes, and if it has metastasized to other organs. Different staging systems are used for various types of cancers.

In [16]:
chunks = []
async for chunk in after_rag_chain.astream("give me a brief about History of Cancer Treatments: Surgery?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 The history of cancer treatments through surgery can be traced back to the late 19th century with the work of surgeons like Bilroth in Germany, Handley in London, and Halsted in Baltimore. Halsted, specifically, developed the radical mastectomy for breast cancer based on the belief that cancer spreads primarily by local invasion rather than through the bloodstream. This belief led to the removal of entire tumors along with nearby lymph nodes, a practice that remained standard for almost a century. However, modern clinical trials in the 1970s demonstrated that less extensive surgery is equally effective for most women with breast cancer, leading to the decline of radical mastectomies and the rise of modified radical mastectomies and lumpectomies with radiation therapy.

Additionally, during this time, surgeons like Handley and Halsted were developing radical operations while Stephen Paget was asking questions about metastasis. Paget's hypothesis that cells spread throughout the body bu

In [9]:
chunks = []
async for chunk in after_rag_chain.astream("what is Genes and Cancer?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 Genes are segments of DNA that contain the instructions for the production of a specific protein or RNA molecule. In the context of cancer, genes can be altered or mutated in various ways, leading to the development and growth of cancer cells. These genetic changes can be caused by various factors such as inherited gene mutations, environmental exposures, viruses, or errors during DNA replication. Understanding the role of genes in cancer is crucial for developing effective prevention strategies, early detection methods, and targeted treatments. Genomic testing, which involves analyzing the genetic material of cancer cells, plays a significant role in identifying these genetic changes and guiding personalized treatment plans.

In [10]:
chunks = []
async for chunk in after_rag_chain.astream("what is alpha blocker?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 An alpha blocker is a type of medication used to treat certain medical conditions, including high blood pressure and urinary symptoms caused by an enlarged prostate (a condition called benign prostatic hyperplasia or BPH). Alpha blockers work by blocking the action of alpha-adrenergic receptors in the body, which helps relax smooth muscles in the blood vessels and the bladder neck. This can lead to improved blood flow and reduced urinary symptoms. Examples of alpha blockers include phentolamine, phenylephrine, timolol, brimonidine, and doxazosin.

In [15]:
chunks = []
async for chunk in after_rag_chain.astream("what is analog?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 An analog is a substance that is similar, but not identical to another substance. For example, a man-made version of a naturally occurring substance.

In [19]:
chunks = []
async for chunk in after_rag_chain.astream("what is anesthesia and its types given in the context?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 Anesthesia refers to the use of medications or techniques to prevent pain during a medical procedure. In the provided context, there are mentions of two types of anesthesia:

1. Local anesthesia: This type of anesthesia numbs a specific area of the body, blocking the transmission of pain signals to the brain. It is typically administered through injections or creams applied directly to the skin. Examples include lidocaine and novocaine.
2. General anesthesia: This type of anesthesia renders the patient unconscious during a surgical procedure, preventing them from feeling pain or being aware of their surroundings. It is typically administered through intravenous (IV) lines or inhalation of gases. Examples include propofol and sevoflurane.

It's important to note that there are other types of anesthesia as well, such as regional anesthesia (which numbs a larger area than local anesthesia but leaves the patient conscious) and monitored anesthesia care (MAC; which involves administering s

In [24]:
chunks = []
async for chunk in after_rag_chain.astream("what is anesthesia and its types?"):
    chunks.append(chunk)
    print(chunk, end="", flush=True)

 Anesthesia refers to the use of medications or other methods to prevent pain during a medical procedure. The types mentioned in the context are:

1. Local anesthesia: numbs a specific area of the body, allowing for the performance of a surgical procedure without causing pain to that area.
2. Regional anesthesia: numbs a larger area than local anesthesia, often used for surgeries on limbs or abdominal procedures.
3. General anesthesia: puts the patient in a deep sleep, making them unaware of the procedure and unable to feel pain throughout their body.

In [25]:
# print(chunks)

In [26]:
# topic = {"question": " What is the difference between a benign and a malignant tumor?"}

# for chunks in after_rag_chain.stream(topic):
#     print(chunks)

In [27]:
# after_rag_chain.invoke("What Is Cancer?")